In [2]:
from pyresparser import extract_text, extract_resume
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

# Extracting data from the resume.pdf file
data = extract_resume("/Users/maitrygajjar/Desktop/Untitled Folder/Deedy_CV-5.pdf")

print("Name:", data["name"])
print("Email:", data["email"])
print("Mobile Number:", data["mobile_number"])
print("Skills:", data["skills"])
print("College Name:", data["college_name"])
print("Degree:", data["degree"])
print("Designation:", data["designation"])
print("Company Names:", data["company_names"])
print("No Of Pages:", data["no_of_pages"])
print("Total Experience:", data["total_experience"])


ImportError: cannot import name 'extract_text' from 'pyresparser' (/Users/maitrygajjar/anaconda3/lib/python3.11/site-packages/pyresparser/__init__.py)

In [6]:
import os
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from pdfminer.high_level import extract_text

# Define the Job and predict_jobs functions
class Job:
    def _init_(self, title, skills):
        self.title = title
        self.skills = set(skills)

def predict_jobs(skill_set, jobs, min_compatibility=50):
    suitable_jobs = []
    for job in jobs:
        # Calculate the number of matching skills
        matching_skills = len(job.skills.intersection(skill_set))
        # Calculate the percentage of matching skills
        match_percentage = (matching_skills / len(job.skills)) * 100
        # Consider a match if the match percentage is at least the specified minimum compatibility
        if match_percentage >= min_compatibility:
            suitable_jobs.append((job.title, match_percentage))
    return suitable_jobs

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

# Constants
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
EMAIL_REG = re.compile(r'[a-z0-9\.\-+]+@[a-z0-9\.\-+]+\.[a-z]+')
SKILLS_DB = [
    'machine learning', 'data science', 'python', 'word', 'excel', 'english', 'JavaScript', 'HTML', 'CSS', 'SQL', 'Java', 'C++', 'R', 'TensorFlow', 'PyTorch', 'Natural Language Processing', 'Computer Vision', 'Git', 'Linux', 'AWS', 'Azure', 'Google Cloud Platform', 'Docker', 'Kubernetes', 'Big Data', 'Spark', 'Hadoop', 'Blockchain', 'Cybersecurity', 'Web Development', 'Mobile App Development', 'UI/UX Design', 'Arduino', 'Raspberry Pi',
    'React', 'Angular', 'Vue.js', 'Node.js', 'Express.js', 'MongoDB', 'PostgreSQL', 'MySQL', 'NoSQL', 'RESTful API', 'GraphQL', 'Microservices', 'DevOps', 'Continuous Integration/Continuous Deployment (CI/CD)', 'Jenkins', 'Ansible', 'Terraform', 'Serverless Architecture', 'IoT Development', 'Cloud Computing', 'Virtualization', 'Containerization', 'Network Administration', 'Shell Scripting', 'Bash', 'PowerShell', 'Ruby', 'Perl', 'PHP', 'Swift', 'Kotlin', 'Flutter',
    'AI-ML[Python]', 'Teamwork and Collaboration', 'Complex Problem-Solving',
    'Leadership', 'Project Management', 'Time Management', 'Communication Skills', 'Adaptability', 'Critical Thinking', 'Emotional Intelligence', 'Decision-Making', 'Conflict Resolution', 'Negotiation', 'Strategic Planning', 'Agile Methodology', 'Scrum', 'Kanban', 'Change Management', 'Risk Management', 'Budgeting', 'Data Analysis', 'Statistical Analysis', 'Market Research', 'Customer Service', 'Sales', 'Marketing', 'Public Speaking', 'Presentation Skills'
]
RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
]

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

# Function to extract phone number
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
    if phone:
        number = ''.join(phone[0])
        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

# Function to extract emails
def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

# Function to extract skills
# Function to extract skills
def extract_skills(resume_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(resume_text)
    filtered_tokens = [w for w in word_tokens if w not in stop_words and w.isalpha()]
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
    found_skills = set()
    for token in filtered_tokens:
        # Case-insensitive matching for skills
        if token.lower() in [skill.lower() for skill in SKILLS_DB]:
            found_skills.add(token)
    for ngram in bigrams_trigrams:
        # Case-insensitive matching for n-grams
        if ngram.lower() in [skill.lower() for skill in SKILLS_DB]:
            found_skills.add(ngram)
    return found_skills


# Function to extract education information
def extract_education(resume_text):
    organizations = []
    for sent in nltk.sent_tokenize(resume_text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join(c[0] for c in chunk.leaves()))
    education = set()
    for org in organizations:
        for word in RESERVED_WORDS:
            if org.lower().find(word) >= 0:
                education.add(org)
    return education

# Function to process a single resume
def process_resume(pdf_path, jobs):
    text = extract_text_from_pdf(pdf_path)
    phone_number = extract_phone_number(text)
    emails = extract_emails(text)
    skills = extract_skills(text)
    education = extract_education(text)
    
    # Predict suitable jobs
    predicted_jobs = predict_jobs(skills, jobs)
    
    return {
        "phone_number": phone_number,
        "emails": emails,
        "skills": skills,
        "education": education,
        "predicted_jobs": predicted_jobs
    }

if _name_ = _main_:
    # Define sample jobs and their required skills
    # Define some sample jobs and their required skills
    jobs = [
    Job("Software Developer", ["Python", "Java", "SQL", "JavaScript", "HTML", "CSS"]),
    Job("Data Analyst", ["SQL", "Excel", "Python", "R", "Data Analysis", "Statistics"]),
    Job("Web Designer", ["HTML", "CSS", "JavaScript", "UI/UX Design"]),
    Job("Network Engineer", ["Networking", "TCP/IP", "Cisco", "Security", "Network Administration"]),
    Job("Systems Administrator", ["Linux", "Windows", "Networking", "Scripting", "Shell Scripting"]),
    Job("UX/UI Designer", ["User Experience", "User Interface", "UI/UX Design", "Adobe XD", "Sketch"]),
    Job("Database Administrator", ["SQL", "Database Management", "Backup and Recovery", "Performance Tuning", "PostgreSQL", "MySQL"]),
    Job("Digital Marketing Specialist", ["SEO", "SEM", "Social Media Marketing", "Content Creation", "Marketing"]),
    Job("Software Engineer", ["Java", "C++", "Python", "Software Development", "Teamwork and Collaboration"]),
    Job("Data Scientist", ["Python", "R", "Machine Learning", "Statistics", "Data Science", "AI-ML[Python]"]),
    Job("Product Manager", ["Product Management", "Project Management", "Agile", "Market Research", "Leadership"]),
    Job("Project Manager", ["Project Management", "Leadership", "Risk Management", "Communication", "Project Management"]),
    Job("Full Stack Developer", ["JavaScript", "HTML", "CSS", "Node.js", "React", "MongoDB"]),
    Job("Frontend Developer", ["JavaScript", "HTML", "CSS", "React", "Angular", "Vue.js"]),
    Job("Backend Developer", ["Java", "Python", "Node.js", "Ruby", "C#", "RESTful API"]),
    Job("DevOps Engineer", ["Linux", "Docker", "Kubernetes", "CI/CD", "Automation", "DevOps"]),
    Job("System Administrator", ["Linux", "Windows", "Networking", "Scripting", "System Administration"]),
    Job("Business Analyst", ["Business Analysis", "Requirement Gathering", "Data Analysis", "Documentation", "Critical Thinking"]),
    Job("QA Engineer", ["Manual Testing", "Automation Testing", "Selenium", "Test Planning", "QA"]),
    Job("Technical Writer", ["Technical Writing", "Documentation", "Editing", "Communication", "Writing"]),
    Job("Cybersecurity Analyst", ["Cybersecurity", "Security Operations", "Incident Response", "Penetration Testing", "Security"]),
    Job("IT Manager", ["IT Management", "Leadership", "Budgeting", "Project Management", "Strategic Planning"]),
    Job("Machine Learning Engineer", ["Python", "Machine Learning", "Deep Learning", "TensorFlow", "PyTorch"]),
    Job("AI Researcher", ["Artificial Intelligence", "Machine Learning", "Natural Language Processing", "Research", "AI-ML[Python]"]),
    Job("Cloud Architect", ["Cloud Computing", "AWS", "Azure", "Google Cloud", "Architecture", "Cloud Computing"]),
    Job("Blockchain Developer", ["Blockchain", "Smart Contracts", "Solidity", "Decentralized Applications", "Blockchain"]),
    Job("Mobile App Developer", ["Android", "iOS", "Java", "Swift", "React Native", "Mobile App Development"]),
    Job("Game Developer", ["Game Development", "Unity", "C#", "C++", "Game Development"]),
    Job("Embedded Systems Engineer", ["Embedded Systems", "Microcontrollers", "C/C++", "RTOS", "Embedded Systems"]),
    Job("IoT Specialist", ["Internet of Things", "Sensors", "Embedded Systems", "Networking", "IoT Development"]),
    Job("Data Engineer", ["SQL", "ETL", "Data Warehousing", "Big Data", "Apache Spark", "Data Engineering"]),
    Job("Statistician", ["Statistics", "R", "Python", "Data Analysis", "Statistical Analysis"]),
    Job("Sales Engineer", ["Sales", "Technical Knowledge", "Customer Relations", "Product Demonstration", "Sales"]),
    Job("Customer Success Manager", ["Customer Success", "Account Management", "Communication", "Problem-Solving", "Customer Service"]),
    Job("Marketing Manager", ["Marketing Strategy", "Digital Marketing", "Market Research", "Campaign Management", "Marketing"]),
    Job("HR Manager", ["Human Resources", "Recruitment", "Employee Relations", "Performance Management", "HR"]),
    Job("Financial Analyst", ["Financial Analysis", "Excel", "Financial Modeling", "Data Analysis", "Finance"]),
    Job("Legal Counsel", ["Legal", "Contract Law", "Corporate Law", "Legal Writing", "Legal"])
]

    
    # Process a single resume
    pdf_path = r"/Users/maitrygajjar/Desktop/Untitled Folder/Om Prakash Karmacharya1.pdf"  # Update with the path to your PDF resume
    result = process_resume(pdf_path, jobs)

    # Display the extracted information and predicted jobs
    print("Phone Number:", result["phone_number"])
    print("Emails:", result["emails"])
    print("Skills:", result["skills"])
    print("Education:", result["education"])
    
    # Display predicted jobs with compatibility percentage
    predicted_jobs = result["predicted_jobs"]
    if predicted_jobs:
        print("\nPredicted suitable jobs based on the provided skill set (with at least 50% match):")
        for job, compatibility in predicted_jobs:
            print(f"- {job}: {compatibility:.2f}% compatibility")
    else:
        print("No suitable jobs found based on the provided skill set.")

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (395835453.py, line 136)

In [7]:
import os
import re
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from pdfminer.high_level import extract_text

# Define the Job and predict_jobs functions
class Job:
    def __init__(self, title, skills):
        self.title = title
        self.skills = set(skills)

def predict_jobs(skill_set, jobs, min_compatibility=50):
    suitable_jobs = []
    for job in jobs:
        # Calculate the number of matching skills
        matching_skills = len(job.skills.intersection(skill_set))
        # Calculate the percentage of matching skills
        match_percentage = (matching_skills / len(job.skills)) * 100
        # Consider a match if the match percentage is at least the specified minimum compatibility
        if match_percentage >= min_compatibility:
            suitable_jobs.append((job.title, match_percentage))
    return suitable_jobs

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

# Constants
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
EMAIL_REG = re.compile(r'[a-z0-9\.\-+]+@[a-z0-9\.\-+]+\.[a-z]+')
SKILLS_DB = [
    'machine learning', 'data science', 'python', 'word', 'excel', 'english', 'JavaScript', 'HTML', 'CSS', 'SQL', 'Java', 'C++', 'R', 'TensorFlow', 'PyTorch', 'Natural Language Processing', 'Computer Vision', 'Git', 'Linux', 'AWS', 'Azure', 'Google Cloud Platform', 'Docker', 'Kubernetes', 'Big Data', 'Spark', 'Hadoop', 'Blockchain', 'Cybersecurity', 'Web Development', 'Mobile App Development', 'UI/UX Design', 'Arduino', 'Raspberry Pi',
    'React', 'Angular', 'Vue.js', 'Node.js', 'Express.js', 'MongoDB', 'PostgreSQL', 'MySQL', 'NoSQL', 'RESTful API', 'GraphQL', 'Microservices', 'DevOps', 'Continuous Integration/Continuous Deployment (CI/CD)', 'Jenkins', 'Ansible', 'Terraform', 'Serverless Architecture', 'IoT Development', 'Cloud Computing', 'Virtualization', 'Containerization', 'Network Administration', 'Shell Scripting', 'Bash', 'PowerShell', 'Ruby', 'Perl', 'PHP', 'Swift', 'Kotlin', 'Flutter',
    'AI-ML[Python]', 'Teamwork and Collaboration', 'Complex Problem-Solving',
    'Leadership', 'Project Management', 'Time Management', 'Communication Skills', 'Adaptability', 'Critical Thinking', 'Emotional Intelligence', 'Decision-Making', 'Conflict Resolution', 'Negotiation', 'Strategic Planning', 'Agile Methodology', 'Scrum', 'Kanban', 'Change Management', 'Risk Management', 'Budgeting', 'Data Analysis', 'Statistical Analysis', 'Market Research', 'Customer Service', 'Sales', 'Marketing', 'Public Speaking', 'Presentation Skills'
]
RESERVED_WORDS = [
    'school',
    'college',
    'univers',
    'academy',
    'faculty',
    'institute',
    'faculdades',
    'schola',
    'schule',
    'lise',
    'lyceum',
    'lycee',
    'polytechnic',
    'kolej',
    'ünivers',
    'okul',
]

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

# Function to extract phone number
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
    if phone:
        number = ''.join(phone[0])
        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

# Function to extract emails
def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

# Function to extract skills
def extract_skills(resume_text):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(resume_text)
    filtered_tokens = [w for w in word_tokens if w not in stop_words and w.isalpha()]
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))
    found_skills = set()
    for token in filtered_tokens:
        # Case-insensitive matching for skills
        if token.lower() in [skill.lower() for skill in SKILLS_DB]:
            found_skills.add(token)
    for ngram in bigrams_trigrams:
        # Case-insensitive matching for n-grams
        if ngram.lower() in [skill.lower() for skill in SKILLS_DB]:
            found_skills.add(ngram)
    return found_skills

# Function to extract education information
def extract_education(resume_text):
    organizations = []
    for sent in nltk.sent_tokenize(resume_text):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
                organizations.append(' '.join(c[0] for c in chunk.leaves()))
    education = set()
    for org in organizations:
        for word in RESERVED_WORDS:
            if org.lower().find(word) >= 0:
                education.add(org)
    return education

# Function to process a single resume
def process_resume(pdf_path, jobs):
    text = extract_text_from_pdf(pdf_path)
    phone_number = extract_phone_number(text)
    emails = extract_emails(text)
    skills = extract_skills(text)
    education = extract_education(text)
    
    # Predict suitable jobs
    predicted_jobs = predict_jobs(skills, jobs)
    
    return {
        "phone_number": phone_number,
        "emails": emails,
        "skills": skills,
        "education": education,
        "predicted_jobs": predicted_jobs
    }

if __name__ == '__main__':
    # Define sample jobs and their required skills
    jobs = [
        Job("Software Developer", ["Python", "Java", "SQL", "JavaScript", "HTML", "CSS"]),
        Job("Data Analyst", ["SQL", "Excel", "Python", "R", "Data Analysis", "Statistics"]),
        Job("Web Designer", ["HTML", "CSS", "JavaScript", "UI/UX Design"]),
        Job("Network Engineer", ["Networking", "TCP/IP", "Cisco", "Security", "Network Administration"]),
        Job("Systems Administrator", ["Linux", "Windows", "Networking", "Scripting", "Shell Scripting"]),
        Job("UX/UI Designer", ["User Experience", "User Interface", "UI/UX Design", "Adobe XD", "Sketch"]),
        Job("Database Administrator", ["SQL", "Database Management", "Backup and Recovery", "Performance Tuning", "PostgreSQL", "MySQL"]),
        Job("Digital Marketing Specialist", ["SEO", "SEM", "Social Media Marketing", "Content Creation", "Marketing"]),
        Job("Software Engineer", ["Java", "C++", "Python", "Software Development", "Teamwork and Collaboration"]),
        Job("Data Scientist", ["Python", "R", "Machine Learning", "Statistics", "Data Science", "AI-ML[Python]"]),
        Job("Product Manager", ["Product Management", "Project Management", "Agile", "Market Research", "Leadership"]),
        Job("Project Manager", ["Project Management", "Leadership", "Risk Management", "Communication", "Project Management"]),
        Job("Full Stack Developer", ["JavaScript", "HTML", "CSS", "Node.js", "React", "MongoDB"]),
        Job("Frontend Developer", ["JavaScript", "HTML", "CSS", "React", "Angular", "Vue.js"]),
        Job("Backend Developer", ["Java", "Python", "Node.js", "Ruby", "C#", "RESTful API"]),
        Job("DevOps Engineer", ["Linux", "Docker", "Kubernetes", "CI/CD", "Automation", "DevOps"]),
        Job("System Administrator", ["Linux", "Windows", "Networking", "Scripting", "System Administration"]),
        Job("Business Analyst", ["Business Analysis", "Requirement Gathering", "Data Analysis", "Documentation", "Critical Thinking"]),
        Job("QA Engineer", ["Manual Testing", "Automation Testing", "Selenium", "Test Planning", "QA"]),
        Job("Technical Writer", ["Technical Writing", "Documentation", "Editing", "Communication", "Writing"]),
        Job("Cybersecurity Analyst", ["Cybersecurity", "Security Operations", "Incident Response", "Penetration Testing", "Security"]),
        Job("IT Manager", ["IT Management", "Leadership", "Budgeting", "Project Management", "Strategic Planning"]),
        Job("Machine Learning Engineer", ["Python", "Machine Learning", "Deep Learning", "TensorFlow", "PyTorch"]),
        Job("AI Researcher", ["Artificial Intelligence", "Machine Learning", "Natural Language Processing", "Research", "AI-ML[Python]"]),
        Job("Cloud Architect", ["Cloud Computing", "AWS", "Azure", "Google Cloud", "Architecture", "Cloud Computing"]),
        Job("Blockchain Developer", ["Blockchain", "Smart Contracts", "Solidity", "Decentralized Applications", "Blockchain"]),
        Job("Mobile App Developer", ["Android", "iOS", "Java", "Swift", "React Native", "Mobile App Development"]),
        Job("Game Developer", ["Game Development", "Unity", "C#", "C++", "Game Development"]),
        Job("Embedded Systems Engineer", ["Embedded Systems", "Microcontrollers", "C/C++", "RTOS", "Embedded Systems"]),
        Job("IoT Specialist", ["Internet of Things", "Sensors", "Embedded Systems", "Networking", "IoT Development"]),
        Job("Data Engineer", ["SQL", "ETL", "Data Warehousing", "Big Data", "Apache Spark", "Data Engineering"]),
        Job("Statistician", ["Statistics", "R", "Python", "Data Analysis", "Statistical Analysis"]),
        Job("Sales Engineer", ["Sales", "Technical Knowledge", "Customer Relations", "Product Demonstration", "Sales"]),
        Job("Customer Success Manager", ["Customer Success", "Account Management", "Communication", "Problem-Solving", "Customer Service"]),
        Job("Marketing Manager", ["Marketing Strategy", "Digital Marketing", "Market Research", "Campaign Management", "Marketing"]),
        Job("HR Manager", ["Human Resources", "Recruitment", "Employee Relations", "Performance Management", "HR"]),
        Job("Financial Analyst", ["Financial Analysis", "Excel", "Financial Modeling", "Data Analysis", "Finance"]),
        Job("Legal Counsel", ["Legal", "Contract Law", "Corporate Law", "Legal Writing", "Legal"])
    ]

    # Process a single resume
    pdf_path = r"/Users/maitrygajjar/Desktop/Untitled Folder/Om Prakash Karmacharya1.pdf"  # Update with the path to your PDF resume
    result = process_resume(pdf_path, jobs)

    # Display the extracted information and predicted jobs
    print("Phone Number:", result["phone_number"])
    print("Emails:", result["emails"])
    print("Skills:", result["skills"])
    print("Education:", result["education"])
    
    # Display predicted jobs with compatibility percentage
    predicted_jobs = result["predicted_jobs"]
    if predicted_jobs:
        print("\nPredicted suitable jobs based on the provided skill set (with at least 50% match):")
        for job, compatibility in predicted_jobs:
            print(f"- {job}: {compatibility:.2f}% compatibility")
    else:
        print("No suitable jobs found based on the provided skill set.")



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/maitrygajjar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/maitrygajjar/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/maitrygajjar/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/maitrygajjar/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maitrygajjar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Phone Number: (573) 222-0175
Emails: ['ompkarmacharya@gmail.com']
Skills: {'python', 'NoSQL', 'GIT', 'Scrum', 'SQL', 'R', 'Big data', 'Docker', 'Spark', 'data science', 'Azure', 'Shell Scripting', 'Java', 'Hadoop', 'JavaScript', 'Big Data', 'AWS', 'Python'}
Education: {'Kathmandu University Career'}

Predicted suitable jobs based on the provided skill set (with at least 50% match):
- Software Developer: 66.67% compatibility
- Data Analyst: 50.00% compatibility
